## Dependencies

In [99]:
# SOURCE: https://github.com/hhursev/recipe-scrapers
# scraper library dependencies
from recipe_scrapers import scrape_me
import re

# scraping dependencies
import datetime
import requests
import time
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import collections
import pprint

## Part 1 - saving index links to a list of nested dictionaries

## ATTEMPT 1

In [ ]:
# Saving URL in a variable to pass into the scraper library.
scraper = scrape_me('https://www.simplyrecipes.com/index/')

In [ ]:
# returns a list of dictionaries containting all of the <a> tag attributes from the defined website. The attribute names are assigned as dictionary keys and the 
raw_dict = scraper.links()

In [ ]:
# for loop that iterates through the dictionary
result = []
for d in raw_dict:
    # Iterates through every key in the dictionary
    for key, value in d.items():
        # If the key is not equal to 'href' and the value doesn't contain ''/recipe' then that pair is skipped.
        if key != "href" or "https://www.simplyrecipes.com/recipes/" not in value:
            # Move onto next iteration
            continue
        txt = ''
        # using part of the URL (key value) to determine the key name
        for i in re.findall("recipes/.*", value):
            txt += i
            title = txt.split('/')[1]
            # appending each key and value to a nested dictionary in a list.

            result.append({title: value})

In [ ]:
result

## ATTEMPT 2

In [ ]:
import requests
from bs4 import BeautifulSoup
import pprint

res = requests.get("https://www.simplyrecipes.com/")
soup = BeautifulSoup(res.text,"html.parser")

links = {}

for div in soup.find("div", class_="rnav-menus").find_all("div", class_="rnav-menu"):
    recipe_type = div.find("span").get_text(strip=True)
    links[recipe_type] = [i.find("a")["href"] for i in div.find_all("li")]

pprint.pprint(links)

## Part 2 - code to scrape a recipe's URL.

#### Notes: we can probably implement this code into a for loop and iterate through whatever structure our complete URL list sits within

In [96]:
# boilerplate code to setup Browser functionality during scrape.
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/williamforsyth/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [101]:
# setting URL for requets
resp = requests.get("https://www.simplyrecipes.com/recipes/cooking_for_two_strawberry_almond_oat_smoothie/")

# Initiating HTML parser, using BeautifulSoup
soup = BeautifulSoup(resp.text, "html.parser")

# using soup.find to find <div> tags with a class of "recipe-callout", that contains each URL recipe
div_ = soup.find("div", attrs={"class": "recipe-callout"})

# Firstly finding all H2 classes and setting as the key. Then iterating through every list item class within "recipe-callout" and splitting the text as the value for that key
recipes = {"_".join(div_.find("h2").text.split()):
               [x.text for x in div_.findAll("li", attrs={"class": "ingredient"})]}

In [102]:

recipes

{'Strawberry_Almond_Oat_Smoothie_Recipe': ['1/2 cup uncooked old-fashioned rolled oats',
  '2 cups frozen strawberries',
  '1 cup plain yogurt (regular or Greek, any fat percentage)',
  '1 cup unsweetened vanilla almond milk (or milk of your choice)',
  '1/2 medium banana, fresh or frozen, sliced',
  '1/4 teaspoon pure almond extract',
  '1-2 teaspoons honey (optional)']}